In [18]:
import os
import pandas as pd
import math
from prettytable import PrettyTable
from operator import itemgetter

## Computing number of samples as a function of epsilon and the other way around

In [19]:
def compute_n(c, delta, epsilon, size_of_market):
    return (c / epsilon) * (c / epsilon) * 0.5 * math.log((2 * size_of_market) / delta)

In [20]:
def compute_eps(c, delta, size_of_market, number_of_samples):
    return c * math.sqrt((math.log((2.0 * size_of_market) / delta)) / (2.0 * number_of_samples))

### Testing eps to n and n to eps functions

In [27]:
c = 2
delta = 0.1
epsilon = 0.05
size_of_market = 4224
n = compute_n(c, delta, epsilon, size_of_market)
assert epsilon == compute_eps(c, delta, size_of_market, n)
assert n == compute_n(c, delta, epsilon, size_of_market)

## Compute Statistics from LSVM worlds

In [28]:
results_dir = '../LSVM/worlds_results/'

In [32]:
def compare_ea_eap(world_number):
    # Read EAP data.
    bidders_final_values_df = pd.read_csv(f"{results_dir}word{world_number}/bidders_final_values.csv")
    eap_samples = bidders_final_values_df['n'].sum()
    
    # Compute EA data.
    bidders_summary_df = pd.read_csv(f"{results_dir}word{world_number}/bidders_summary.csv")
    params_csv = pd.read_csv(f"{results_dir}word{world_number}/params.csv")
    c, delta, epsilon = int(params_csv.iloc[3]['value']), sum(eval(params_csv.iloc[1]['value'])), float(params_csv.iloc[6]['value'])
    size_of_market = sum([2 ** len(eval(row[2])) for row in bidders_summary_df.itertuples()])
    ea_samples = compute_n(c, delta, epsilon, size_of_market)
    #print(f"c = {c}, delta = {delta}, epsilon = {epsilon : .4f}, size_of_market = {size_of_market}")
    #print(f"{ ea_samples : ,.0f}")
    #print(f"{ eap_samples : ,}")
    
    return [world_number, 
            c, 
            delta, 
            f"{epsilon : .4f}", 
            f"{size_of_market: ,}", 
            f"{ea_samples : ,.0f}", 
            f"{eap_samples : ,.0f}",
            f"{(eap_samples / ea_samples) * 100.0 : .4f}"]

In [33]:
results_ptable = PrettyTable()
results_ptable.field_names = ['world', 'c', 'delta', 'epsilon', 'size_of_market', 'ea', 'eap', 'ratio']
results_ptable.align["ea"] = "r"
results_ptable.align["eap"] = "r"
results = []
for i in range(0, 100):
    results += [compare_ea_eap(i)]
results = sorted(results, key=itemgetter(7))

for row in results:
    results_ptable.add_row(row)
print(results_ptable)

+-------+---+-------+---------+----------------+---------+----------+-----------+
| world | c | delta | epsilon | size_of_market |      ea |      eap |   ratio   |
+-------+---+-------+---------+----------------+---------+----------+-----------+
|   2   | 2 |  0.1  |  0.0346 |       768      |  16,089 |   58,080 |  360.9974 |
|   11  | 2 |  0.1  |  0.0346 |       448      |  15,189 |   54,880 |  361.3105 |
|   48  | 2 |  0.1  |  0.0346 |       576      |  15,609 |   56,430 |  361.5314 |
|   30  | 2 |  0.1  |  0.0346 |       512      |  15,412 |   55,790 |  361.9902 |
|   59  | 2 |  0.1  |  0.0346 |       832      |  16,222 |   58,810 |  362.5245 |
|   71  | 2 |  0.1  |  0.0346 |       960      |  16,461 |   59,910 |  363.9469 |
|   40  | 2 |  0.1  |  0.0346 |      1,152     |  16,765 |   61,650 |  367.7195 |
|   99  | 2 |  0.1  |  0.0346 |       832      |  16,222 |   60,160 |  370.8463 |
|   62  | 2 |  0.1  |  0.0346 |      1,152     |  16,765 |   62,280 |  371.4772 |
|   42  | 2 |  0